Retention Redshift Insert (API Method)
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* APr 5, 2022

In [1]:
import boto3
import getpass
import pandas as pd
import awswrangler as wr
import pyodbc

from datetime import datetime
from pytz import timezone


### Let's connect to Redshift QA-APP to get our dataset:

In [2]:
! aws sso login --profile Stellaralgo-DataScienceAdmin

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

DBQN-DJNZ
Successully logged into Start URL: https://stellaralgo.awsapps.com/start#/


In [3]:
try:
    session = boto3.setup_default_session(profile_name='Stellaralgo-DataScienceAdmin')
    client = boto3.client('redshift')

    connqa = wr.data_api.redshift.connect(
        cluster_id="qa-app",
        database="datascience",
        db_user="admin"
    )
    
    print('QA CREDENTIALS RETRIEVED SUCCESSFULLY')
    
except Exception as err:
    print('FAILED TO RETRIEVE QA CREDENTIALS!', err)

QA CREDENTIALS RETRIEVED SUCCESSFULLY


In [4]:
qacode =  f"""
    SELECT *
    FROM 
       datascience.ds.teamsconfig
    """
df = wr.data_api.redshift.read_sql_query(
            sql = qacode, 
            con = connqa
        )
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   teamproductyearid  112 non-null    int64 
 1   lkupclientid       112 non-null    int64 
 2   clientcode         112 non-null    object
 3   productgrouping    112 non-null    object
 4   trainseasonyear    112 non-null    int64 
 5   testseasonyear     112 non-null    int64 
 6   leaguename         112 non-null    object
 7   dbname             112 non-null    object
dtypes: int64(4), object(4)
memory usage: 7.1+ KB


### Now we can create another session to RedShift PROD-APP, to inser data:

In [5]:
try:
    session = boto3.setup_default_session(profile_name='Stellaralgo-DataScienceAdmin')
    client = boto3.client('redshift')

    connprod = wr.data_api.redshift.connect(
        cluster_id="prod-app",
        database="datascience",
        db_user="admin"
    )
    
    print('PROD CREDENTIALS RETRIEVED SUCCESSFULLY')
    
except Exception as err:
    print('FAILED TO RETRIEVE PROD CREDENTIALS!', err)

PROD CREDENTIALS RETRIEVED SUCCESSFULLY


### Insert the new scores into the teamsconfig table:

In [14]:
try:
    fields = f"""
    INSERT INTO datascience.ds.teamsconfig (
        teamproductyearid,
        lkupclientid,
        clientcode,
        productgrouping,
        trainseasonyear,
        testseasonyear,
        leaguename,
        dbname
    ) VALUES """

    # Insert Dataframe into SQL Server:
    values_list = []
    for i, row in df.iterrows():
        

        values = f"""
            (
                {str(row.teamproductyearid)},
                {str(row.lkupclientid)},
                '{str(row.clientcode)}',
                '{str(row.productgrouping)}',
                {str(row.trainseasonyear)},
                {str(row.testseasonyear)},
                '{str(row.leaguename)}',
                '{str(row.dbname)}'
            )"""
        if values is not values_list:
            values_list.append(values)
                
    insert_statement = fields + ",".join(values_list)+";"

    wr.data_api.redshift.read_sql_query(
        sql = insert_statement, 
        con = connprod
            )

except Exception as err:
    print("ERROR WHILE INSERTING TEAMS CONFIG TO DATABASE:", err)

print("TEAMS CONFIG INSERTS COMPLETED")

TEAMS CONFIG INSERTS COMPLETED
